In [1]:
import xarray as xr
import numpy as np

from microsim import schema as ms 

### 1. A single white light source

In [2]:
# Define light sources
light_source = ms.LightSource(
    name="white_light",
    spectrum=ms.Spectrum(
        wavelength=np.arange(400, 701, 1),
        intensity=np.ones(301),
    ),
    power=1.0
)

In [3]:
# Create an Optical Config with this light source
config = ms.OpticalConfig(
    lights=[light_source],
    filters=[], 
    name="white_light_no_filter"
)

NOTE: We use no filter, since they do not interfere with the light source at this stage

In [4]:
sim = ms.Simulation(
    truth_space=ms.ShapeScaleSpace(shape=(64, 256, 256), scale=(0.04, 0.02, 0.02)),
    output_space={"downscale": 4},
    sample=ms.Sample(
        labels=[
            ms.FluorophoreDistribution(
                distribution=ms.MatsLines(density=0.5, length=30, azimuth=5, max_r=1),
                fluorophore="EGFP",
            )
        ]
    ),
    channels=[config],
    modality=ms.Confocal(pinhole_au=0.5),
    settings=ms.Settings(
        random_seed=100, 
        max_psf_radius_aus=8,
        binning_strategy='equal_space',
        num_wavelength_bins=32
    ),
    detector=ms.CameraCCD(qe=0.82, read_noise=2, bit_depth=12),
    # output_path="au1.tif",
)

In [6]:
gt = sim.ground_truth()

In [20]:
illum_arr = sim.illumination_flux(gt, channel_idx=0)

In [21]:
illum_arr

<xarray.DataArray (w: 32, c: 1, z: 64, y: 256, x: 256)> Size: 1GB
array([[[[[8., 8., 8., ..., 8., 8., 8.],
          [8., 8., 8., ..., 8., 8., 8.],
          [8., 8., 8., ..., 8., 8., 8.],
          ...,
          [8., 8., 8., ..., 8., 8., 8.],
          [8., 8., 8., ..., 8., 8., 8.],
          [8., 8., 8., ..., 8., 8., 8.]],

         [[8., 8., 8., ..., 8., 8., 8.],
          [8., 8., 8., ..., 8., 8., 8.],
          [8., 8., 8., ..., 8., 8., 8.],
          ...,
          [8., 8., 8., ..., 8., 8., 8.],
          [8., 8., 8., ..., 8., 8., 8.],
          [8., 8., 8., ..., 8., 8., 8.]],

         [[8., 8., 8., ..., 8., 8., 8.],
          [8., 8., 8., ..., 8., 8., 8.],
          [8., 8., 8., ..., 8., 8., 8.],
          ...,
...
          ...,
          [6., 6., 6., ..., 6., 6., 6.],
          [6., 6., 6., ..., 6., 6., 6.],
          [6., 6., 6., ..., 6., 6., 6.]],

         [[6., 6., 6., ..., 6., 6., 6.],
          [6., 6., 6., ..., 6., 6., 6.],
          [6., 6., 6., ..., 6., 6., 6.],
          ...,
          [6., 6., 6., ..., 6., 6., 6.],
          [6., 6., 6., ..., 6., 6., 6.],
          [6., 6., 6., ..., 6., 6., 6.]],

         [[6., 6., 6., ..., 6., 6., 6.],
          [6., 6., 6., ..., 6., 6., 6.],
          [6., 6., 6., ..., 6., 6., 6.],
          ...,
          [6., 6., 6., ..., 6., 6., 6.],
          [6., 6., 6., ..., 6., 6., 6.],
          [6., 6., 6., ..., 6., 6., 6.]]]]])
Coordinates:
  * w        (w) object 256B (463.0, 471.0] (471.0, 479.0] ... (693.0, 699.0]
  * z        (z) float64 512B 0.0 0.04 0.08 0.12 0.16 ... 2.4 2.44 2.48 2.52
  * y        (y) float64 2kB 0.0 0.02 0.04 0.06 0.08 ... 5.02 5.04 5.06 5.08 5.1
  * x        (x) float64 2kB 0.0 0.02 0.04 0.06 0.08 ... 5.02 5.04 5.06 5.08 5.1
  * c        (c) object 8B name='white_light_no_filter' filters=[] lights=[Li...

### Overlapping Light Sources with uniform intensity

NOTE: the resulting light source spectrum is given by the product of the intensities for overlapping wavelengths. Therefore, the resulting light source is 0 if at least 2 of the overlapping intensities are non overlapping.

In [32]:
ls1 = ms.LightSource(
    name="blue_light",
    spectrum=ms.Spectrum(
        wavelength=np.arange(400, 551, 1),
        intensity=np.ones(151),
    ),
    power=1.0
)
ls2 = ms.LightSource(
    name="yellow_light",
    spectrum=ms.Spectrum(
        wavelength=np.arange(500, 601, 1),
        intensity=np.ones(101),
    ),
    power=1.0
)

In [33]:
# Create an Optical Config with this light source
config = ms.OpticalConfig(
    lights=[ls1, ls2],
    filters=[], 
    name="overlapping_uniform_no_filter"
)

In [34]:
sim = ms.Simulation(
    truth_space=ms.ShapeScaleSpace(shape=(64, 256, 256), scale=(0.04, 0.02, 0.02)),
    output_space={"downscale": 4},
    sample=ms.Sample(
        labels=[
            ms.FluorophoreDistribution(
                distribution=ms.MatsLines(density=0.5, length=30, azimuth=5, max_r=1),
                fluorophore="EGFP",
            )
        ]
    ),
    channels=[config],
    modality=ms.Confocal(pinhole_au=0.5),
    settings=ms.Settings(
        random_seed=100, 
        max_psf_radius_aus=8,
        binning_strategy='equal_space',
        num_wavelength_bins=32
    ),
    detector=ms.CameraCCD(qe=0.82, read_noise=2, bit_depth=12),
    # output_path="au1.tif",
)

In [35]:
gt = sim.ground_truth()

In [36]:
illum_arr = sim.illumination_flux(gt, channel_idx=0)

In [37]:
illum_arr

<xarray.DataArray (w: 32, c: 1, z: 64, y: 256, x: 256)> Size: 1GB
array([[[[[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]],

         [[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]],

         [[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
...
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]],

         [[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]],

         [[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]]]]])
Coordinates:
  * w        (w) object 256B (463.0, 471.0] (471.0, 479.0] ... (693.0, 699.0]
  * z        (z) float64 512B 0.0 0.04 0.08 0.12 0.16 ... 2.4 2.44 2.48 2.52
  * y        (y) float64 2kB 0.0 0.02 0.04 0.06 0.08 ... 5.02 5.04 5.06 5.08 5.1
  * x        (x) float64 2kB 0.0 0.02 0.04 0.06 0.08 ... 5.02 5.04 5.06 5.08 5.1
  * c        (c) object 8B name='overlapping_uniform_no_filter' filters=[] li...

In [39]:
illum_arr.mean(dim=("c", "z", "y", "x"))

<xarray.DataArray (w: 32)> Size: 256B
array([nan, nan, nan, nan,  4.,  8.,  8.,  8.,  8.,  8.,  7., nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan])
Coordinates:
  * w        (w) object 256B (463.0, 471.0] (471.0, 479.0] ... (693.0, 699.0]